<h1>Extration of the ECG Features from the Synthetic ECG</h1>

In [ ]:
pip install neurokit2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.5/696.5 kB 12.7 MB/s eta 0:00:00


In [ ]:
# Generate longer ECG signal (e.g., 20 seconds)
import neurokit2 as nk
import pandas as pd
import matplotlib.pyplot as plt

# Longer duration to ensure it's processable
ecg_signal = nk.ecg_simulate(duration=20, sampling_rate=250)

# Save to CSV
df = pd.DataFrame({"ECG": ecg_signal})
df.to_csv("synthetic_ecg.csv", index=False)
print("✅ ECG saved: synthetic_ecg.csv")


✅ ECG saved: synthetic_ecg.csv


In [7]:
import pandas as pd
import neurokit2 as nk
import numpy as np

# Load ECG
df = pd.read_csv("synthetic_ecg.csv")
ecg_signal = df["ECG"]

# Process ECG
signals, info = nk.ecg_process(ecg_signal, sampling_rate=250)

# Extract thalach (max heart rate)
thalach = np.max(signals["ECG_Rate"])

# Approximate ST segment "slope" (custom logic)
# Take signal 50ms after R-peaks and 150ms after, calculate average difference
rpeaks = info["ECG_R_Peaks"]
sampling_rate = 250
st_slope_values = []

for r in rpeaks:
    try:
        start = r + int(0.05 * sampling_rate)
        end = r + int(0.15 * sampling_rate)
        if end < len(ecg_signal):
            slope_val = ecg_signal[end] - ecg_signal[start]
            st_slope_values.append(slope_val)
    except:
        continue

# Compute average slope
slope = np.mean(st_slope_values) if st_slope_values else 0

# Categorize slope:
# 0 = flat, 1 = upsloping, 2 = downsloping
if slope > 0.05:
    slope_category = 1  # upsloping
elif slope < -0.05:
    slope_category = 2  # downsloping
else:
    slope_category = 0  # flat

# Estimate "oldpeak" = signal depression after R-peak vs baseline
baseline = np.mean(ecg_signal[:250])  # use first second as baseline
oldpeak_values = []

for r in rpeaks:
    try:
        offset = r + int(0.1 * sampling_rate)
        if offset < len(ecg_signal):
            oldpeak_values.append(baseline - ecg_signal[offset])
    except:
        continue

oldpeak = np.mean(oldpeak_values) if oldpeak_values else 0

# Final extracted features
features = {
    "thalach": thalach,
    "slope": slope_category,
    "oldpeak": oldpeak
}

print("✅ Extracted ECG Features:")
for key, val in features.items():
    print(f"{key}: {val:.2f}" if key != "slope" else f"{key}: {val}")


✅ Extracted ECG Features:
thalach: 72.12
slope: 1
oldpeak: 0.12
